In [1]:
if (!require(forecast)){
    installed <- T
    install.packages("forecast")
}
if (!require(dplyr)){
    install.packages("dplyr")
    require(dplyr)
}

Loading required package: forecast
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
arima_prediction <- function(defect_backlog_path, weeks_to_predict_path, forecast_summary_path){

defect_backlog = read.csv(file=defect_backlog_path, header=TRUE, sep=",")
weeks_to_predict = read.csv(file=weeks_to_predict_path, header=TRUE, sep=",")
    
weekly_inflow_ts = ts(defect_backlog$inflow_all)
numbers_of_weeks_to_predict = weeks_to_predict$number

forecasts <- c()
actuals <- c()
errors <- c()
method <- "auto_arima"
    
for(i in 1:nrow(weeks_to_predict)){
    model = auto.arima(subset(weekly_inflow_ts, 
                       start = 1,
                       end = numbers_of_weeks_to_predict[i]))
    
    fc = forecast(model,1)
    forecasts[i] = round(fc$mean)
    actuals[i] = defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]) %>% select (inflow_all)
    e <- (1-as.integer(forecasts[i])/as.integer(actuals[i]))*100
    errors[i] = round(abs(e),1)
}

mean_error <- round(mean(errors),2)
result <- cbind(Method = method, Week=numbers_of_weeks_to_predict, Actual=actuals, Forecast=forecasts, Error = errors)

write.table(result, file = forecast_summary_path, sep=",")
}

In [3]:
arima_prediction("/Users/paulina/magisterka/eclipse/converted_data/platform-backlog_30_12_12-4_1_16.csv",
                 "/Users/paulina/magisterka/eclipse/samples/samples_30_12_12-4_1_16.csv",
                 "/Users/paulina/magisterka/eclipse/forecasts/auto_arima_30_12_12-4_1_16.csv")

In [4]:
read.csv(file="/Users/paulina/magisterka/eclipse/forecasts/auto_arima_30_12_12-4_1_16.csv", header=TRUE, sep=",")

Method,Week,Actual,Forecast,Error
auto_arima,16,74,71,4.1
auto_arima,26,54,72,33.3
auto_arima,27,59,71,20.3
auto_arima,33,58,71,22.4
auto_arima,44,66,66,0.0
auto_arima,51,23,58,152.2
auto_arima,62,86,86,0.0
auto_arima,65,88,86,2.3
auto_arima,70,85,74,12.9
auto_arima,83,80,74,7.5
